In [1]:
import os
os.system('pip3 install urllib3==1.26.6 s3fs boto3 google-api-python-client google-auth-httplib2 google-auth-oauthlib')

0

In [2]:
#Importação das bibliotecas do Google Sheets para interagir com as planilhas do Google
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.errors import HttpError

#Interagir com a AWS
import boto3
import json

#### Secrets Manager

In [3]:
#Criando um client com o secretsmanager
client = boto3.client('secretsmanager', region_name='us-east-1')

#Pegando as credenciais da aws
secret = client.get_secret_value(SecretId='user-athena-requets-aws-secrets')

#Convertendo em um objeto JSON
chave = json.loads(secret['SecretString'])

#### Athena

In [4]:
# Este código coleta dados do AWS Athena por meio da API.
# Informações de acesso
AWS_ACCESS_KEY = chave['AWS_ACCESS_KEY']
AWS_SECRET_KEY = chave['AWS_SECRET_KEY']
S3_OUTPUT = "s3://bkt-estudo/Retorno/"
AWS_REGION = "us-east-1"
DATABASE = "nubank"

In [5]:
TABLES = ["despesas","limite","receitas","saldo","transacoes"]

In [6]:
  # Criação do cliente do Athena
client = boto3.client('athena',
                      aws_access_key_id=AWS_ACCESS_KEY,
                      aws_secret_access_key=AWS_SECRET_KEY,
                      region_name=AWS_REGION)

In [7]:
# Para cada tabela, inicia uma execução de consulta.
for x in TABLES:
    # A string de consulta seleciona todas as linhas da tabela.
    response = client.start_query_execution(
        QueryString=f'SELECT * FROM "{DATABASE}"."{x}"',
        QueryExecutionContext={
            'Database': DATABASE
        },
        ResultConfiguration={
            'OutputLocation': S3_OUTPUT
        })

    # Espera até que a consulta termine.
    while True:
        stats = client.get_query_execution(QueryExecutionId=response['QueryExecutionId'])
        status = stats['QueryExecution']['Status']['State']

        if status in ['SUCCEEDED', 'FAILED', 'CANCELLED']:
            break

    # Obtém os resultados da consulta.
    column_data = []
    results = client.get_query_results(QueryExecutionId=response['QueryExecutionId'])

    # Se houver um próximo token, obtém a próxima página de resultados.
    if 'NextToken' in results:
        token = results['NextToken']

        for row in results['ResultSet']['Rows']:
            row_data = [col['VarCharValue'] for col in row['Data']]
            column_data.append(row_data)

        while True:
            data = []
            results = client.get_query_results(QueryExecutionId=response['QueryExecutionId'], NextToken=token)
            for row in results['ResultSet']['Rows']:
                row_data = [col['VarCharValue'] for col in row['Data']]
                data.append(row_data)
            column_data.extend(data[1:])

            if 'NextToken' not in results:
                break  # Sai do loop se não houver mais tokens.

            token = results['NextToken']

    # Caso contrário, não há mais páginas de resultados.
    else:
        for row in results['ResultSet']['Rows']:
            row_data = [col['VarCharValue'] for col in row['Data']]
            column_data.append(row_data)

    # Armazena os dados das colunas em uma variável global.
    globals()[f"{x}"] = column_data

#### S3

In [8]:
s3 = boto3.client('s3')

#Extraindo do S3 as credenciais da API do Sheets
s3.download_file('nubank-api-dependencias', 'token.json', 'token.json')

#### Google Sheets

In [9]:
# Scopes para a API do Google Sheets
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

In [10]:
# ID da planilha e intervalos de células que serão lidos
SAMPLE_SPREADSHEET_ID = '1tLoo0DXR9crNrvzpCsaRCIvtn3VC1wIHEIpF-v_hkEg'

creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# Cria o objeto de serviço da API do Google Sheets
service = build('sheets', 'v4', credentials=creds)
sheet = service.spreadsheets()

In [11]:
#Intervalos e abas da planilha
ss_transacoes = ['Transacao!A2:A','Transacao!B2:B','Transacao!C2:C','Transacao!D2:D','Transacao!E2:E','Transacao!F2:F','Transacao!G2:G']
ss_receitas = ['Receitas!A2:A','Receitas!B2:B','Receitas!C2:C']
ss_despesas = ['Despesas!A2:A','Despesas!B2:B','Despesas!C2:C']

In [12]:
def transformar_lista(lista):
    resultado = [[item] for item in lista]
    return resultado

#### Enviando os dados para a Planilha

##### Transacoes

In [35]:
def remover_tres_zeros(data_string):
    lista = []
    for x in data_string:
        lista.append(x[:-4])
    return lista

In [36]:
#Envian para a planilha as informacoes sobre transacoes
datas = remover_tres_zeros([x[3] for x in transacoes[1:]])
categoria = [x[1] for x in transacoes[1:]]
descricao = [x[2] for x in transacoes[1:]]
tipo = [x[4] for x in transacoes[1:]]
valor = [x[0].replace('.', ',') for x in transacoes[1:]]
qtd_parcelas = [x[5] for x in transacoes[1:]]
parcela = [x[6] for x in transacoes[1:]]
l = [datas,categoria,descricao,tipo,valor,qtd_parcelas,parcela]

for a,b in zip(l,ss_transacoes):
  sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                        range=b, valueInputOption='USER_ENTERED',
                        body={'values':transformar_lista(a)}).execute()

###### Receitas

In [37]:
#Mandando para a planilha as informacoes sobre receitas
datass = remover_tres_zeros([x[0] for x in receitas[1:]])
descricaos = [x[1] for x in receitas[1:]]
valors = [x[2].replace('.', ',') for x in receitas[1:]]
ls = [datass,descricaos,valors]

for c,d in zip(ls,ss_receitas):
  sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                        range=d, valueInputOption='USER_ENTERED',
                        body={'values':transformar_lista(c)}).execute()

##### Despesas

In [38]:
#Mandando para a planilha as informacoes sobre despesas
datasss = remover_tres_zeros([x[0] for x in despesas[1:]])
descricaoss = [x[1] for x in despesas[1:]]
valorss = [x[2].replace('.', ',') for x in despesas[1:]]
lss = [datasss,descricaoss,valorss]

for e,f in zip(lss,ss_despesas):
  sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                        range=f, valueInputOption='USER_ENTERED',
                        body={'values':transformar_lista(e)}).execute()

##### Limite

In [39]:
ss_limite = ['Limite!A2:A','Limite!B2:B','Limite!C2:C','Limite!D2:D']

In [40]:
#Mandando para a planilha as informacoes sobre o limite
datassss = remover_tres_zeros([x[0] for x in limite[1:]])
fatura_atual = [x[1].replace('.', ',') for x in limite[1:]]
limite_disponivel = [x[2].replace('.', ',') for x in limite[1:]]
limitess = [x[3].replace('.', ',') for x in limite[1:]]
lsss = [datassss,fatura_atual,limite_disponivel,limitess]

for g,h in zip(lsss,ss_limite):
  sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                        range=h, valueInputOption='USER_ENTERED',
                        body={'values':transformar_lista(g)}).execute()

##### Saldo

In [41]:
ss_saldo = ['Saldo!A2:A','Saldo!B2:B']

In [42]:
#Mandando para a planilha as informacoes sobre o saldo
datasssss = remover_tres_zeros([x[0] for x in saldo[1:]])
saldos = [x[1].replace('.', ',') for x in saldo[1:]]
lssss = [datasssss,saldos]

for i,j in zip(lssss,ss_saldo):
  sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                        range=j, valueInputOption='USER_ENTERED',
                        body={'values':transformar_lista(i)}).execute()